### QUESTÃO 3
### Realize um Holdout com 1-NN (distância Euclidiana), utilizando 70% dos dados para treinamento e o restante (30%) para teste na base de dados Wine archive.ics.uci. edu/ml/datasets/Wine. Você precisa mostrar como calculou cada métrica, não pode utilizar biblioteca que já calcula a métrica diretamente mas pode utilizar biblioteca para o 1-NN e para dividir os dados entre treino e teste. Dica: pode utilizar as seguintes bibliotecas.

### (a) Calcule a matriz de confusão. 

In [49]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# MATRIZ DE CONFUSÃO
def calc_confusion_matrix_n(y_teste, y_pred):
    
    confusion_matrix_n = []
    matrix_build = []
    list_class_uniq = y_teste.Class.unique()

    # construção de matriz zerada
    for x in range(len(list_class_uniq)):    
        for y in range(len(list_class_uniq)):
            matrix_build.append(0)

        confusion_matrix_n.append(matrix_build)
        matrix_build = []

    for x in range(len(y_pred)):
        confusion_matrix_n[y_teste.Class.to_numpy()[x]-1][y_pred[x]-1] += 1

    count = 1

    print("MATRIZ DE CONFUSÃO \n")
    # COLUNA
    print("  1   2   3")

    for x in confusion_matrix_n:
        print(str(count) + ' ' + str(x))
        count += 1

    return confusion_matrix_n

# RECALL
def calc_recall_class(confusion_matrix_n):
    recall_current = 0
    vp = 0
    fn = 0
    recall = []

    for i in range(len(confusion_matrix_n)):
        for x in range(len(confusion_matrix_n[i])):
            if x == recall_current:
                vp = confusion_matrix_n[i][x]
            else:
                fn += confusion_matrix_n[i][x]
        
        recall_current += 1

        # FUNÇÃO DO CALCULO DE RECALL
        cal_recall = (vp / (vp + fn))
        fn = 0
        
        recall.append(cal_recall)

    count = 1

    print ("RECALL \n")
    for K in recall:
        print( count, format(K, '.1f'))
        count += 1
    
    return recall

# TAXA DE ACERTO 
def calc_hit_rate(confusion_matrix_n):
    true = 0
    v = 0
    total = 0

    for i in range(len(confusion_matrix_n)):
        for x in range(len(confusion_matrix_n[i])):
            if x == true:
                v += confusion_matrix_n[i][x]
            total += confusion_matrix_n[i][x]    
        
        true += 1

    hit_rate = (v / (total))

    print("TAXA DE ACERTO \n")
    print(format(hit_rate, '.2f'))

    return format(hit_rate, '.2f')

# PRECISÃO
def calc_precision(confusion_matrix_n):
    current_accuracy = 0
    vp = 0
    fp = 0
    precision = []

    for i in range(len(confusion_matrix_n)):
        for x in range(len(confusion_matrix_n[i])):
            if x == current_accuracy:
                vp = confusion_matrix_n[i][x]
            else:
                fp += confusion_matrix_n[x][i]
        
        current_accuracy += 1
        calculo = (vp / (vp + fp))
        fp = 0
        
        precision.append(calculo)

    count = 1
    print ("PRECISÃO \n")
    for y in precision:
        print( count, format(y, '.2f'))
        count += 1

    return precision

# Medida-F
def calc_measure_f(precision, recall):
    f = []
    for i in range(len(precision)):
        
        f.append((2*precision[i]*recall[i])/(precision[i]+recall[i]))

    count = 1
    print("MEDIDA F \n")
    for x in f:
        print(count, format(x, '.2f'))
        count += 1

    return f

# TAXA DE FP
def calc_fp_rate(confusion_matrix_n):
    flag = 0
    fp = 0
    vp = 0
    vp_array = []
    fp_array = []
    fp_rate = []

    for p in range(len(confusion_matrix_n)):
        for t in range(len(confusion_matrix_n)):
            for k in range(len(confusion_matrix_n[t])):
                if t != p and k != p:
                    vp += confusion_matrix_n[k][t]

        vp_array.append(vp)
        vp = 0

    for i in range(len(confusion_matrix_n)):
        flag = 0
        for x in range(len(confusion_matrix_n[i])):
            if i != flag:
                fp += confusion_matrix_n[x][i]
            flag += 1               
        
        fp_array.append(fp)
        fp = 0
        
    for j in range(len(fp_array)):
        fp_rate.append(fp_array[j] / (vp_array[j] + fp_array[j]))


    count = 1
    print("TAXA DE FP \n")
    for y in fp_rate:
        print( count, format(y, '.2f'))
        count += 1

    return fp_rate

# ------------------------------------ DADOS --------------------------------------

names_columns = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

df = pd.read_csv("wine.data", names=names_columns)
df.Alcohol.unique()

df = pd.read_csv("wine.data", names=names_columns)
df.Alcohol.unique()

classe_1 = df[(df['Class'] == 1)]
classe_2 = df[(df['Class'] == 2)]
classe_3 = df[(df['Class'] == 3)]

# DIVISÃO DE 30% TESTE
classe_1_treino, classe_1_teste = train_test_split(classe_1,test_size=0.3)
classe_2_treino, classe_2_teste = train_test_split(classe_2,test_size=0.3)
classe_3_treino, classe_3_teste = train_test_split(classe_3,test_size=0.3)

# treino
treino = pd.concat([classe_1_treino, classe_2_treino, classe_3_treino])
x_treino = treino[treino.columns[1:13:]]
y_treino = treino[treino.columns[:1:]]

# teste
teste = pd.concat([classe_1_teste, classe_2_teste, classe_3_teste])
x_teste = teste[teste.columns[1:13:]]
y_teste = teste[teste.columns[:1:]]

scaler = StandardScaler()
scaler.fit(x_treino)

# MANTER DENTRO DE UMA ESCALA 
x_treino = scaler.transform(x_treino)
x_teste = scaler.transform(x_teste)

# 1-NN
classificador = KNeighborsClassifier(n_neighbors=1)
classificador.fit(x_treino, y_treino)

y_pred = classificador.predict(x_teste)

# ------------------------------------ CHAMADA DE FUNÇÕES --------------------------------------

print("\n (a) Calcule a matriz de confusão: \n")
confusion_matrix_n = calc_confusion_matrix_n(y_teste, y_pred)

print("\n (b) Calcule o Recall por classe \n")
recall = calc_recall_class(confusion_matrix_n)

print("\n (c) Calcule a Taxa de acerto do classificador \n")
calc_hit_rate(confusion_matrix_n)

print("\n (d) Calcule a Precisão por classe \n")
precision = calc_precision(confusion_matrix_n)

print("\n (e) Calcule a Medida-F por classe \n")
calc_measure_f(precision, recall)

print("\n (f) Calcule a Taxa de FP por classe \n")
fp_rate = calc_fp_rate(confusion_matrix_n)


 (a) Calcule a matriz de confusão: 

MATRIZ DE CONFUSÃO 

  1   2   3
1 [17, 1, 0]
2 [1, 19, 2]
3 [0, 0, 15]

 (b) Calcule o Recall por classe 

RECALL 

1 0.9
2 0.9
3 1.0

 (c) Calcule a Taxa de acerto do classificador 

TAXA DE ACERTO 

0.93

 (d) Calcule a Precisão por classe 

PRECISÃO 

1 0.94
2 0.95
3 0.88

 (e) Calcule a Medida-F por classe 

MEDIDA F 

1 0.94
2 0.90
3 0.94

 (f) Calcule a Taxa de FP por classe 

TAXA DE FP 

1 0.03
2 0.03
3 0.05


/home/laisy/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


### Compare com os resultados da questão anterior com resultados das métricas computados a partir de uma biblioteca que já calcula diretamente estas métricas. Dica: você pode utilizar sklearn.metrics. https://scikit-learn.org/stable/modules/classes.html# module-sklearn.metrics 


In [52]:
print("\n (a) Calcule a matriz de confusão: \n")
print(confusion_matrix(y_teste, y_pred))

print("\n (b) Calcule o Recall por classe \n")

print("\n (c) Calcule a Taxa de acerto do classificador \n")

print("\n (d) Calcule a Precisão por classe \n")

print(classification_report(y_teste, y_pred))

print("\n (e) Calcule a Medida-F por classe \n")

print("\n (f) Calcule a Taxa de FP por classe \n")

fp = confusion_matrix(y_teste, y_pred).sum(axis=0) - np.diag(confusion_matrix(y_teste, y_pred))  
fn = confusion_matrix(y_teste, y_pred).sum(axis=1) - np.diag(confusion_matrix(y_teste, y_pred))
tp = np.diag(confusion_matrix(y_teste, y_pred))
tn = confusion_matrix(y_teste, y_pred).sum() - (fp + fn + tp) 

fp = fp.astype(float)
tn = tn.astype(float)

fp_rate = fp/(fp+tn)

count = 1
print ("fp_rate")
for y in fp_rate:
    print(format(y, '.2f'))
    count += 1



 (a) Calcule a matriz de confusão: 

[[17  1  0]
 [ 1 19  2]
 [ 0  0 15]]

 (b) Calcule o Recall por classe 


 (c) Calcule a Taxa de acerto do classificador 


 (d) Calcule a Precisão por classe 

              precision    recall  f1-score   support

           1       0.94      0.94      0.94        18
           2       0.95      0.86      0.90        22
           3       0.88      1.00      0.94        15

    accuracy                           0.93        55
   macro avg       0.93      0.94      0.93        55
weighted avg       0.93      0.93      0.93        55


 (e) Calcule a Medida-F por classe 


 (f) Calcule a Taxa de FP por classe 

fp_rate
0.03
0.03
0.05
